# Using a HF model (variant 1)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "mistralai/Mistral-7B-Instruct-v0.2"
tok = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")

messages = [
    {"role": "system", "content": "Du bist ein deutscher Assistent zur Klassifikation von Anfragen."},
    {"role": "user", "content": "Wieviele Semester dauert das Studium Künstliche Intelligenz und Computer Vision?"},
]

prompt = tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

out = model.generate(**tok(prompt, return_tensors="pt").to(model.device), max_new_tokens=50)
print(tok.decode(out[0], skip_special_tokens=True))


In [ ]:
type(tok)

In [ ]:
messages

In [ ]:
prompt

In [ ]:
prompt2 = tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)

In [ ]:
prompt2

In [ ]:
len(prompt2)

In [ ]:
tok(prompt2, return_tensors="pt")

In [ ]:
out

In [ ]:
out[0]

In [ ]:
tok.decode(out[0], skip_special_tokens=True)

# Using a HF model (variant 2)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda" # the device to load the model onto

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")


messages = [
    {"role": "user", "content": "What is your favourite condiment?"},
    {"role": "assistant", "content": "Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!"},
    {"role": "user", "content": "Do you have mayonnaise recipes?"}
]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

model_inputs = encodeds.to(device)
model.to(device)

generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])


# Using a HF model (variant 3: with quantization)

In [ ]:
!pip install bitsandbytes

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "mistralai/Mistral-7B-Instruct-v0.2"

bnb_cfg = BitsAndBytesConfig(
    load_in_4bit=True,                 # 4-bit Quantisierung
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",         # gute Wahl für LLMs
    bnb_4bit_compute_dtype=torch.bfloat16  # Rechen-Dtype
)

tok = AutoTokenizer.from_pretrained(model_id)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_cfg,
    torch_dtype=torch.bfloat16,
    device_map="auto"                  # sharded auf GPU/CPU verteilen
)

messages = [
    {"role": "system", "content": "Du bist ein deutscher Assistent zur Klassifikation von Anfragen."},
    {"role": "user", "content": "Wieviele Semester dauert das Studium Künstliche Intelligenz und Computer Vision an der Hochschule Kempten?"},
]

inputs = tok.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True)

# WICHTIG: Nicht model.to("cuda") aufrufen. Das Modell ist bereits sharded+quantisiert.
# Entweder Inputs auf "cuda" legen...
inputs = tok.apply_chat_template(
    messages, return_tensors="pt", add_generation_prompt=True
).to("cuda")  # einfach den Tensor verschieben

out = model.generate(
    input_ids=inputs,             # explizit benennen
    max_new_tokens=256,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    pad_token_id=tok.pad_token_id
)

print(tok.decode(out[0], skip_special_tokens=True))

`torch_dtype` is deprecated! Use `dtype` instead!
2025-11-05 11:40:58.528210: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-05 11:40:58.566264: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-05 11:40:59.461652: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


[INST] Du bist ein deutscher Assistent zur Klassifikation von Anfragen.

Wieviele Semester dauert das Studium Künstliche Intelligenz und Computer Vision an der Hochschule Kempten? [/INST] In Deutschland variiert die Dauer des Studiums Künstliche Intelligenz und Computer Vision an verschiedenen Hochschulen. An der Hochschule Kempten dauert das Studium der Masterstudiengänge Computer Science mit Schwerpunkt Computer Vision und Kognitive Systeme, das auch Aspekte der Künstlichen Intelligenz behandelt, in der Regel sechs Semester. Das entspricht in der Regel zwei Jahren, aber es kann je nach Einzelheiten des Studienplans etwas länger dauern.

Für das Bachelorstudium gibt es an der Hochschule Kempten keinen direkt vergleichbaren Studiengang, der die genauen Inhalte von Künstlicher Intelligenz und Computer Vision umfasst. Es gibt jedoch Studiengänge wie Informatik, die Grundkenntnisse in diesen Bereichen vermitteln und auf die ein Masterstudium folgen kann. Das Bachelorstudium dauert in der 

In [ ]:
inputs

# Anfragen-Klassifikation mit einem HF Modell

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "mistralai/Mistral-7B-Instruct-v0.2"

bnb_cfg = BitsAndBytesConfig(
    load_in_4bit=True,                 # 4-bit Quantisierung
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",         # gute Wahl für LLMs
    bnb_4bit_compute_dtype=torch.bfloat16  # Rechen-Dtype
)

tok = AutoTokenizer.from_pretrained(model_id)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_cfg,
    torch_dtype=torch.bfloat16,
    device_map="auto"                  # sharded auf GPU/CPU verteilen
)


anfrage1 = """
            Wieviele Semester dauert das Studium Künstliche Intelligenz und Computer Vision?
          """

anfrage2 = """
            Ohh!! So viel zu tun! Brauche mehr Zeit bis zur Abgabe meiner Masterarbeit!
            Können Sie die Frist bitte einen Monat nach hinten schieben!
          """

anfrage3 = """
            Wir sind die Firma SuperSki3000. Wir suchen Studenten als Skilehrer.
            Dürften wir am Eingang hierzu ein Poster bei Ihnen aufhängen?
           """

anfrage4 = """
            Darf ich das Busticket eigentlich auch in den Semesterferien nutzen?
           """

anfrage5 = """
            Was gibt es heute in der Mensa zu essen?
           """

anfrage6 = """
            Darf ich mein Auto auch am Wochenende in der Tiefgarage parken?
           """

anfragen = [anfrage1,
            anfrage2,
            anfrage3,
            anfrage4,
            anfrage5,
            anfrage6 ]

for anfrage in anfragen:

  prompt = f"""
            Du bist ein Assistent zur Klassifikation von Benutzeranfragen auf der
            Webseite der Hochschule Kempten.

            Hier die Anfrage:
            ---
            {anfrage}
            ---

            Du sollst jede Anfrage klassifizieren in eine der folgenden 5 Klassen:
            ---
            1. Anfrage zu einem speziellem Studium
            2. Antrag auf Fristverlängerung für Abschlussarbeit
            3. Anfrage wegen Veröffentlichung Stellenangebote
            4. Anfrage wegen Semesterticket
            5. Anfrage Sonstiges
            ---
            Antworte nur mit 1-5
          """
  
  print("*" * 100)
  print(prompt)


  messages = [      
      {"role": "user", "content": prompt},
  ]

  inputs = tok.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True)

  # WICHTIG: Nicht model.to("cuda") aufrufen. Das Modell ist bereits sharded+quantisiert.
  # Entweder Inputs auf "cuda" legen...
  inputs = tok.apply_chat_template(
      messages, return_tensors="pt", add_generation_prompt=True
  ).to("cuda")  # einfach den Tensor verschieben

  
  out = model.generate(
      input_ids=inputs,             # explizit benennen
      max_new_tokens=256,
      do_sample=True,
      temperature=0.7,
      top_p=0.9,
      pad_token_id=tok.pad_token_id
  )

  resp = tok.decode(out[0], skip_special_tokens=True)
  print(f"Das LLM klassifiziert diese Anfrage als: {resp}")
  print("*" * 100)
  print("\n\n\n")

`torch_dtype` is deprecated! Use `dtype` instead!
2025-11-05 12:01:42.064352: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-05 12:01:42.101327: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-05 12:01:43.004053: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


****************************************************************************************************

            Du bist ein Assistent zur Klassifikation von Benutzeranfragen auf der
            Webseite der Hochschule Kempten.

            Hier die Anfrage:
            ---
            
            Wieviele Semester dauert das Studium Künstliche Intelligenz und Computer Vision?
          
            ---

            Du sollst jede Anfrage klassifizieren in eine der folgenden 5 Klassen:
            ---
            1. Anfrage zu einem speziellem Studium
            2. Antrag auf Fristverlängerung für Abschlussarbeit
            3. Anfrage wegen Veröffentlichung Stellenangebote
            4. Anfrage wegen Semesterticket
            5. Anfrage Sonstiges
            ---
            Antworte nur mit 1-5
          
Das LLM klassifiziert diese Anfrage als: [INST] 
            Du bist ein Assistent zur Klassifikation von Benutzeranfragen auf der
            Webseite der Hochschule Kempten.



# Textsegmentierung

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "mistralai/Mistral-7B-Instruct-v0.2"

bnb_cfg = BitsAndBytesConfig(
    load_in_4bit=True,                 # 4-bit Quantisierung
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",         # gute Wahl für LLMs
    bnb_4bit_compute_dtype=torch.bfloat16  # Rechen-Dtype
)

tok = AutoTokenizer.from_pretrained(model_id)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_cfg,
    torch_dtype=torch.bfloat16,
    device_map="auto"                  # sharded auf GPU/CPU verteilen
)


txt1 = """
            Ich bin zum Kunden nach Oberstaufen gefahren. Im Transformatorraum
            roch es verschmort und einer der Adapter war auch zu heiß.
            Der Adapter X17 war kaputt. Habe ihn deswegen durch
            einen neuen Adapter vom Typ X19-200 ausgetauscht und habe die
            Steuerung der Seilbahn noch hoch gefahren. Jetzt läuft die Seilbahn
            wieder perfekt! Habe in der Kantine vom Kunden gegessen. Dort gab es leckere
            Spaghetti Bolognese! Viele Grüße, Hans!
          """

texte = [txt1]

for txt in texte:

  prompt = f"""
            Du bist ein Assistent der in Freitextfeldern von technischen Außendienstmitarbeitern
            die Sätze identifizieren soll, die beschreiben,
            1. was für ein Problem beim Kunden vorlag
            2. was der Außendienstmitarbeiter für eine Lösung durchgeführt hat
            
            Hier der Text des Außendienstmitarbeiters
            ---
            {txt}
            ---

            Du sollst jeden Text segmentieren und folgenden Output erzeugen:
            ---
            Problembeschreibung: <alle Sätze, die das Problem beschreiben>
            Lösungsbeschreibung: <alle Sätze, die die Lösung beschreiben>
            Irrelevantes:  <alle Sätze, die weder das Problem oder die Lösung beschreiben>
            ---            
          """
  
  print("*" * 100)
  print(prompt)


  messages = [      
      {"role": "user", "content": prompt},
  ]

  inputs = tok.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True)

  # WICHTIG: Nicht model.to("cuda") aufrufen. Das Modell ist bereits sharded+quantisiert.
  # Entweder Inputs auf "cuda" legen...
  inputs = tok.apply_chat_template(
      messages, return_tensors="pt", add_generation_prompt=True
  ).to("cuda")  # einfach den Tensor verschieben

  
  out = model.generate(
      input_ids=inputs,             # explizit benennen
      max_new_tokens=256,
      do_sample=True,
      temperature=0.7,
      top_p=0.9,
      pad_token_id=tok.pad_token_id
  )

  resp = tok.decode(out[0], skip_special_tokens=True)
  print(f"LLM AUSGABE: {resp}")
  print("*" * 100)
  print("\n\n\n")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

****************************************************************************************************

            Du bist ein Assistent der in Freitextfeldern von technischen Außendienstmitarbeitern
            die Sätze identifizieren soll, die beschreiben,
            1. was für ein Problem beim Kunden vorlag
            2. was der Außendienstmitarbeiter für eine Lösung durchgeführt hat

            Hier der Text des Außendienstmitarbeiters
            ---
            
            Ich bin zum Kunden nach Oberstaufen gefahren. Im Transformatorraum
            roch es verschmort und einer der Adapter war auch zu heiß.
            Der Adapter X17 war kaputt. Habe ihn deswegen durch
            einen neuen Adapter vom Typ X19-200 ausgetauscht und habe die
            Steuerung der Seilbahn noch hoch gefahren. Jetzt läuft die Seilbahn
            wieder perfekt! Habe in der Kantine vom Kunden gegessen. Dort gab es leckere
            Spaghetti Bolognese! Viele Grüße, Hans!
          
    